## Install `simple_salesforce`  module

Remove the comment symbol (#) to run `!pip install simple_salesforce`. Once this module is installed, you can comment it out again. 

In [1]:
#!pip install simple_salesforce

## Import dependencies

In [2]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
import numpy as np
pymysql.install_as_MySQLdb()

## Import configuration variables

In [3]:
# import sys
# sys.path.append('../../../../')
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

## Set up `simple_salesforce`

Pass in the username, password, and security token here. The security token contains information about your org.

In [4]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

## Connect to your MySQL database

In [5]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Query Data

In [6]:
staff_assignment_data_df = pd.read_sql("SELECT * FROM staffassignment", conn)
staff_assignment_data_df.head(30)

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


## CREATE Course LookUp table #1 from Class Table

In [7]:
query=sf.query_all_iter("SELECT ID_Class__c, Name FROM Class__c")

In [8]:
query

<generator object Salesforce.query_all_iter at 0x00000183A43563C8>

In [9]:
class_lookup_list = []

for rec in query:
    
    sf_record = {
        'Class_lookup_ID':rec['Name'],
        'ID_Class': rec['ID_Class__c']
              
    }
    class_lookup_list.append(sf_record)  

In [10]:
class_lookup_list

[{'Class_lookup_ID': 'a0B3h000001jxIK', 'ID_Class': '1'},
 {'Class_lookup_ID': 'a0B3h000001jxIP', 'ID_Class': '2'},
 {'Class_lookup_ID': 'a0B3h000001jxIZ', 'ID_Class': '4'},
 {'Class_lookup_ID': 'a0B3h000001jxIU', 'ID_Class': '3'}]

In [11]:
class_lookup_df = pd.DataFrame(class_lookup_list)

class_lookup_df.head(20)

,Class_lookup_ID,ID_Class
0,a0B3h000001jxIK,1
1,a0B3h000001jxIP,2
2,a0B3h000001jxIZ,4
3,a0B3h000001jxIU,3


In [12]:
class_lookup_df.dtypes

Class_lookup_ID    object
ID_Class           object
dtype: object

In [13]:
class_lookup_df["Class_lookup_ID"]=class_lookup_df["Class_lookup_ID"].astype(str)
class_lookup_df["ID_Class"]=class_lookup_df["ID_Class"].astype(str)

In [14]:
class_lookup_df.dtypes

Class_lookup_ID    object
ID_Class           object
dtype: object

## CREATE Course LookUp table #2 from Staff Table


In [15]:
query=sf.query_all_iter("SELECT ID_Staff__c, Name FROM Staff__c")

In [16]:
query

<generator object Salesforce.query_all_iter at 0x00000183A4379748>

In [17]:
staff_lookup_list = []

for rec in query:
    
    sf_record = {
        'Staff_lookup_ID':rec['Name'],
        'ID_Staff': rec['ID_Staff__c']
              
    }
    staff_lookup_list.append(sf_record)  

In [18]:
staff_lookup_list

[{'Staff_lookup_ID': 'a0D3h000001hcYM', 'ID_Staff': 3.0},
 {'Staff_lookup_ID': 'a0D3h000001hcYR', 'ID_Staff': 4.0},
 {'Staff_lookup_ID': 'a0D3h000001hcYH', 'ID_Staff': 2.0},
 {'Staff_lookup_ID': 'a0D3h000001hcYC', 'ID_Staff': 1.0}]

In [19]:
staff_lookup_df = pd.DataFrame(staff_lookup_list)

staff_lookup_df.head(5)

,Staff_lookup_ID,ID_Staff
0,a0D3h000001hcYM,3.0
1,a0D3h000001hcYR,4.0
2,a0D3h000001hcYH,2.0
3,a0D3h000001hcYC,1.0


In [20]:
staff_lookup_df.dtypes

Staff_lookup_ID     object
ID_Staff           float64
dtype: object

In [21]:
new = np.array(staff_lookup_df["ID_Staff"])

In [22]:
staff_lookup_df["ID_Staff"]=new.astype(int)

In [23]:
staff_lookup_df["ID_Staff"]

0    3
1    4
2    2
3    1
Name: ID_Staff, dtype: int32

In [24]:
staff_lookup_df["ID_Staff"]=staff_lookup_df["ID_Staff"].astype(str)

In [25]:
staff_lookup_df.head()

,Staff_lookup_ID,ID_Staff
0,a0D3h000001hcYM,3
1,a0D3h000001hcYR,4
2,a0D3h000001hcYH,2
3,a0D3h000001hcYC,1


## Joining the tables


In [26]:
query='''
    SELECT 
        sa.*
    FROM
        staffassignment sa
        INNER JOIN class c
        ON c.ID_Class=sa.ID_Class
        INNER JOIN staff i
        ON sa.ID_Staff=i.ID_Staff
'''

staff_assignment_data_df=pd.read_sql(query, conn)
staff_assignment_data_df.head()

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [27]:
staff_assignment_data_df.rename(columns={
    'Role':'Role__c',
    'StartDate':'StartDate__c',
    'EndDate':'EndDate__c',

}, inplace=True)

staff_assignment_data_df.head(5)

,ID_StaffAssignment,ID_Staff,ID_Class,Role__c,StartDate__c,EndDate__c
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [28]:
staff_assignment_data_df.dtypes

ID_StaffAssignment     int64
ID_Staff               int64
ID_Class               int64
Role__c               object
StartDate__c          object
EndDate__c            object
dtype: object

In [29]:
staff_assignment_data_df.astype({'ID_StaffAssignment': 'str'}).dtypes

ID_StaffAssignment    object
ID_Staff               int64
ID_Class               int64
Role__c               object
StartDate__c          object
EndDate__c            object
dtype: object

In [30]:
staff_assignment_data_df["ID_StaffAssignment"]=staff_assignment_data_df["ID_StaffAssignment"].astype(str)
staff_assignment_data_df["ID_Staff"]=staff_assignment_data_df["ID_Staff"].astype(str)
staff_assignment_data_df["ID_Class"]=staff_assignment_data_df["ID_Class"].astype(str)


In [31]:
staff_assignment_data_df.dtypes

ID_StaffAssignment    object
ID_Staff              object
ID_Class              object
Role__c               object
StartDate__c          object
EndDate__c            object
dtype: object

In [32]:
staff_assignment_data_df1 = pd.merge(staff_assignment_data_df,class_lookup_df,how='left')
staff_assignment_data_df1.head(5)


,ID_StaffAssignment,ID_Staff,ID_Class,Role__c,StartDate__c,EndDate__c,Class_lookup_ID
0,1,1,1,Teacher Assistant,2020-03-16,None,a0B3h000001jxIK
1,2,4,1,Teacher Assistant,2020-03-16,None,a0B3h000001jxIK
2,3,3,1,Instructor,2020-03-16,None,a0B3h000001jxIK


In [33]:
staff_assignment_data_df2 = pd.merge(staff_assignment_data_df1,staff_lookup_df,how='left')
staff_assignment_data_df2.head()
staff_assignment_data_df2.fillna(0)

,ID_StaffAssignment,ID_Staff,ID_Class,Role__c,StartDate__c,EndDate__c,Class_lookup_ID,Staff_lookup_ID
0,1,1,1,Teacher Assistant,2020-03-16,0,a0B3h000001jxIK,a0D3h000001hcYC
1,2,4,1,Teacher Assistant,2020-03-16,0,a0B3h000001jxIK,a0D3h000001hcYR
2,3,3,1,Instructor,2020-03-16,0,a0B3h000001jxIK,a0D3h000001hcYM


In [34]:
staff_assignment_data_df2['StartDate__c'] = pd.to_datetime(staff_assignment_data_df2['StartDate__c']).dt.date
staff_assignment_data_df2['EndDate__c'] = pd.to_datetime(staff_assignment_data_df2['EndDate__c']).dt.date
staff_assignment_data_df2.drop(columns = ['EndDate__c'], inplace=True)
staff_assignment_data_df2.head()

,ID_StaffAssignment,ID_Staff,ID_Class,Role__c,StartDate__c,Class_lookup_ID,Staff_lookup_ID
0,1,1,1,Teacher Assistant,2020-03-16,a0B3h000001jxIK,a0D3h000001hcYC
1,2,4,1,Teacher Assistant,2020-03-16,a0B3h000001jxIK,a0D3h000001hcYR
2,3,3,1,Instructor,2020-03-16,a0B3h000001jxIK,a0D3h000001hcYM


In [35]:
staff_assignment_data_records = staff_assignment_data_df2.to_dict(orient='records')
staff_assignment_data_records

[{'ID_StaffAssignment': '1',
  'ID_Staff': '1',
  'ID_Class': '1',
  'Role__c': 'Teacher Assistant',
  'StartDate__c': datetime.date(2020, 3, 16),
  'Class_lookup_ID': 'a0B3h000001jxIK',
  'Staff_lookup_ID': 'a0D3h000001hcYC'},
 {'ID_StaffAssignment': '2',
  'ID_Staff': '4',
  'ID_Class': '1',
  'Role__c': 'Teacher Assistant',
  'StartDate__c': datetime.date(2020, 3, 16),
  'Class_lookup_ID': 'a0B3h000001jxIK',
  'Staff_lookup_ID': 'a0D3h000001hcYR'},
 {'ID_StaffAssignment': '3',
  'ID_Staff': '3',
  'ID_Class': '1',
  'Role__c': 'Instructor',
  'StartDate__c': datetime.date(2020, 3, 16),
  'Class_lookup_ID': 'a0B3h000001jxIK',
  'Staff_lookup_ID': 'a0D3h000001hcYM'}]

In [36]:
staff_assignment_data_df2.dtypes

ID_StaffAssignment    object
ID_Staff              object
ID_Class              object
Role__c               object
StartDate__c          object
Class_lookup_ID       object
Staff_lookup_ID       object
dtype: object

In [40]:
for rec in staff_assignment_data_records:
 
    record = {
        'ID_StaffAssignment__c':rec['ID_StaffAssignment'],
        'ID_Staff__c':rec['ID_Staff'],
        'ID_Class__c': rec['ID_Class'],
        'Role__c': rec['Role__c'],
        'StartDate__c': str(rec['StartDate__c']),
        'Class__c':rec['Class_lookup_ID'],
        'Staff__c':rec['Staff_lookup_ID'],
        
    }
    
    try:
        sf.Staff_Assignment__c.create(record)
    except Exception as e:
        print(e) 